In [3]:
from pyspark import SparkConf, SparkContext

In [4]:
conf = SparkConf().setMaster("local").setAppName("bus-stop")
sc = SparkContext(conf=conf)

22/10/15 01:45:00 WARN Utils: Your hostname, imhaneul-ui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 192.168.0.18 instead (on interface en0)
22/10/15 01:45:00 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/10/15 01:45:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [55]:
import os

directory = f"{os.getcwd()}/data"
csv_file = "bus.csv"

data = sc.textFile(f"file:///{directory}/{csv_file}")
header = data.first()
filetred_lines = data.filter(lambda row: row != header)
lines_split = filetred_lines.map(lambda x: x.split(",")[-1])

In [58]:
count = lines_split.countByValue()

In [59]:
import pandas as pd 
pd.Series(count).to_csv("city_count.csv", header=None)

In [60]:
data = pd.read_csv("city_count.csv", names=["city", "count"])
data

,city,count
0,태백시,434
1,삼척시,801
2,속초시,203
3,동해시,301
4,영월군,639
...,...,...
122,포천시,1166
123,여주시,1215
124,가평군,928
125,양평군,1102
